# Example(Feature Enginnering WIne)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
df = pd.read_csv('./Wine_red.csv',sep=';')
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


In [3]:
features = df.drop('quality',axis=1)
targets = df.quality

In [4]:
#engineer interaction features
ploy = PolynomialFeatures(2,interaction_only=True,include_bias=False)
features_e = ploy.fit_transform(features)
cols = ploy.get_feature_names(features.columns)
features_e = pd.DataFrame(features_e,columns=cols)
print('features shape',features.shape)
print('features_e shape',features_e.shape)
features_e.head(3)

features shape (1599, 11)
features_e shape (1599, 66)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,...,total sulfur dioxide density,total sulfur dioxide pH,total sulfur dioxide sulphates,total sulfur dioxide alcohol,density pH,density sulphates,density alcohol,pH sulphates,pH alcohol,sulphates alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,...,33.9252,119.34,19.04,319.6,3.502278,0.558768,9.37932,1.9656,32.994,5.264
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,...,66.7856,214.40,45.56,656.6,3.189760,0.677824,9.76864,2.1760,31.360,6.664
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,...,53.8380,176.04,35.10,529.2,3.250220,0.648050,9.77060,2.1190,31.948,6.370


In [5]:
#standardize features
features_e = (features_e - features_e.mean())/features_e.std()

In [6]:
features_e['bias']=1
features_e.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,...,total sulfur dioxide pH,total sulfur dioxide sulphates,total sulfur dioxide alcohol,density pH,density sulphates,density alcohol,pH sulphates,pH alcohol,sulphates alcohol,bias
0,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,...,-0.318888,-0.466169,-0.469886,1.330644,-0.575015,-0.959307,-0.393794,-0.368454,-0.837776,1
1,-0.298454,1.966827,-1.391037,0.043403,0.223805,0.872365,0.624168,0.028252,-0.719708,0.128910,...,0.567921,0.583123,0.535341,-0.727649,0.128665,-0.589072,0.003660,-0.756503,-0.110527,1
2,-0.298454,1.296660,-1.185699,-0.169374,0.096323,-0.083643,0.228975,0.134222,-0.331073,-0.048074,...,0.210063,0.169262,0.155323,-0.329450,-0.047314,-0.587208,-0.104015,-0.616863,-0.263249,1
3,1.654339,-1.384011,1.483689,-0.453077,-0.264878,0.107558,0.411372,0.664069,-0.978798,-0.461036,...,0.336563,0.157392,0.330716,-0.965278,-0.456379,-0.577888,-0.644658,-0.849597,-0.619601,1
4,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,...,-0.318888,-0.466169,-0.469886,1.330644,-0.575015,-0.959307,-0.393794,-0.368454,-0.837776,1


In [7]:
features_e.describe().T[['mean','std']]

,mean,std
fixed acidity,-1.570643e-14,1.0
volatile acidity,-1.590973e-15,1.0
citric acid,6.045057e-15,1.0
residual sugar,-1.344696e-15,1.0
chlorides,5.098590e-15,1.0
...,...,...
density alcohol,3.305882e-15,1.0
pH sulphates,-7.825944e-15,1.0
pH alcohol,-7.732054e-15,1.0
sulphates alcohol,1.840287e-15,1.0


In [8]:
# cross-validation R2 bigger better
lr = LinearRegression(fit_intercept=False)
cv_results = cross_validate(lr,features_e,targets,
                            return_train_score=True)
R2_train = cv_results['train_score'].mean()
R2_test = cv_results['test_score'].mean()
print('train R2',R2_train.round(3))
print('test R2',R2_test.round(3))

train R2 0.435
test R2 0.247


In [10]:
#coefficients
lr.fit(features_e,targets)
coef = pd.Series(lr.coef_,index=features_e.columns)
coef.sort_values(ascending=False)

total sulfur dioxide            111.950530
free sulfur dioxide density      89.493996
sulphates                        64.908481
chlorides density                59.382580
citric acid density              30.382760
                                   ...    
citric acid                     -27.516352
chlorides                       -58.332904
density sulphates               -66.543408
free sulfur dioxide             -88.119447
total sulfur dioxide density   -113.412922
Length: 67, dtype: float64